In [1]:
import pickle
from pymetheus.pymetheus import LogicNet

In [2]:
vectors = pickle.load(open('vectors.pickle', 'rb'))
fr = vectors['FR'][:5]
it = vectors['IT'][:5]

In [3]:
it[0]["uri"]

'Abano_Terme'

In [4]:
ll = LogicNet()
ll.predicate("country", argument_size=200)
ll.constant("France", definition=vectors['France']['vec'])
ll.constant("Italy", definition=vectors['Italy']['vec'])
for f in fr:
    ll.constant(f['uri'], definition=f['vec'])
for i in it:
    ll.constant(i['uri'], definition=i['vec'])

for f in fr:
    for i in it:
        ll.knowledge(f"country({f['uri']},France)")
        ll.knowledge(f"~country({f['uri']},Italy)")
        ll.knowledge(f"country({i['uri']},Italy)")
        ll.knowledge(f"~country({i['uri']},France)")

ll.variable("?a", (list(map(lambda x: x['uri'], fr)) + list(map(lambda x: x['uri'], it))))
ll.variable("?b", (list(map(lambda x: x['uri'], fr)) + list(map(lambda x: x['uri'], it))))
rule_3 = "forall ?a,?b: country(?a,?b) -> ~country(?b,?a)"
rule_4 = "forall ?a: ~country(?a,?a)"
#ll.universal_rule(rule_3)
#ll.universal_rule(rule_4)

In [5]:
ll.fit(epochs=2000, grouping=99)


Current Satisfiability 0.874264):  57%|█████▋    | 1135/2000 [00:15<00:12, 71.20it/s]

KeyboardInterrupt: 

In [ ]:
ll.constant('Parma', definition=vectors['IT'][10]['vec'], overwrite=True)
ll.constant('Le_Mans', definition=vectors['FR'][10]['vec'], overwrite=True)
ll.reason("~country(Parma,France)", True)
ll.reason("country(Parma,Italy)", True)
ll.reason("~country(Le_Mans,France)", True)
ll.reason("country(Le_Mans,Italy)", True)
ll.reason("~country(Parma, Le_Mans)",True)

In [ ]:
ll.reason(rule_4)

Current Satisfiability 0.500000):  89%|████████▉ | 1788/2000 [00:29<00:01, 113.84it/s]

In [5]:
cloudpickle.dump(ll, open('network.pickle', 'wb'))

In [ ]:
ll2 = cloudpickle.load(open('network.pickle', 'rb'))
ll2.reason('capital(Rome)')